# Data-Prep

**gefiltert wird:**
- duplikate entferenen
- min 100 Posts
- min 50 Replies
- Emoji only posts
- nur links
- reposts
- weniger als 3 wörter

In [3]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import re
import os
import uuid
import re
import emoji 


In [5]:
path = "data/raw/Kopie von FolloweeIDs2_tweets_df_AugustPull.csv"

dtypes = {
    'tweet_id': str,  # Read as string for easier map keying
    'original_user_id': str, # Assuming user IDs can be treated as strings
    'reply_to_id': str # Read as string, will handle 'nan' or empty strings
}
df = pd.read_csv(path, sep=",", encoding="utf-8", dtype=dtypes, low_memory=False, on_bad_lines='skip')


df.columns


Index(['full_text', 'tweet_id', 'created_at', 'screen_name',
       'original_user_id', 'retweeted_user_ID', 'collected_at', 'reply_to_id',
       'reply_to_user', 'expandedURL'],
      dtype='object')

In [6]:
def is_only_emojis(text):
    """
    Prüft, ob ein Text nur aus Emojis und Leerzeichen besteht.
    Diese Version ist mit neueren Versionen der 'emoji'-Bibliothek kompatibel.
    """
    if not isinstance(text, str) or not text.strip():
        return False
    
    # In neueren Versionen der Bibliothek wird emoji.EMOJI_DATA verwendet.
    # Die Keys dieses Dictionaries sind die eigentlichen Emoji-Zeichen.
    all_emojis_set = set(emoji.EMOJI_DATA.keys())
    
    # Entferne alle Emoji-Zeichen aus dem Text
    text_without_emojis = ''.join(char for char in text if char not in all_emojis_set)
    
    # Wenn der Rest nach dem Entfernen von Leerzeichen leer ist, bestand der Text nur aus Emojis
    return not text_without_emojis.strip()


def is_only_link(text):
    """Prüft, ob ein Text nur aus URLs und Leerzeichen besteht."""
    if not isinstance(text, str) or not text.strip():
        return False
    url_pattern = r'https?://[^\s/$.?#].[^\s]*'
    text_without_urls = re.sub(url_pattern, '', text, flags=re.MULTILINE)
    return not text_without_urls.strip()

def analyze_url_patterns(text):
    """Analyze URL patterns in text"""
    if not isinstance(text, str):
        return {"has_urls": False}
    
    url_pattern = r'https?://[^\s/$.?#].[^\s]*'
    urls = re.findall(url_pattern, text)
    
    return {"has_urls": len(urls) > 0}

def should_filter_context_aware(text: str, min_words: int = 5) -> bool:
    """Remove posts with URLs unless they have substantial context"""
    analysis = analyze_url_patterns(text)
    if not analysis["has_urls"]:
        return False
    
    # Remove if not enough meaningful text
    text_without_urls = re.sub(r'https?://[^\s/$.?#].[^\s]*', '', text, flags=re.MULTILINE)
    meaningful_words = len([word for word in text_without_urls.split() if len(word) > 2])
    
    return meaningful_words < min_words

def contains_link(text: str) -> bool:
    """
    Prüft, ob ein Text eine URL enthält.
    Gibt True zurück, wenn mindestens eine URL gefunden wird, sonst False.
    """
    if not isinstance(text, str):
        return False
    
    # Nutzt die bestehende Regex-Logik
    url_pattern = r'https?://[^\s/$.?#].[^\s]*'
    urls = re.findall(url_pattern, text)
    return len(urls) > 0

def has_minimum_words(text, min_words=3):
    """
    Check if text has at least min_words words.
    Returns True if text has enough words, False otherwise.
    """
    if pd.isna(text) or text == '':
        return False
    
    # Split by whitespace and filter out empty strings
    words = [word.strip() for word in str(text).split() if word.strip()]
    return len(words) >= min_words

In [7]:


# --- Configuration Constants ---
TARGET_POSTS_PER_SET = 100  # Total posts in each history/holdout set
TARGET_REPLIES_PER_SET = 50   # Number of replies in each set
TARGET_ORIGINALS_PER_SET = TARGET_POSTS_PER_SET - TARGET_REPLIES_PER_SET

# --- File Path ---
path = "data/raw/Kopie von FolloweeIDs2_tweets_df_AugustPull.csv"

# --- Main Script ---
def process_tweets(csv_path):
    print(f"Loading data from: {csv_path}")

    # Specify dtypes for key ID columns and others prone to mixed types
    # Reading IDs as strings simplifies handling and map keying.
    dtypes = {
        'tweet_id': str,
        'original_user_id': str,
        'reply_to_id': str,
        'screen_name': str, # Was in DtypeWarning
        'created_at': str,  # Was in DtypeWarning, consider parse_dates if needed
        'collected_at': str,# Was in DtypeWarning
        'expandedURL': str  # Was in DtypeWarning
    }
    
    try:
        df = pd.read_csv(
            csv_path, 
            sep=",", 
            encoding="utf-8", 
            dtype=dtypes, 
            low_memory=False, # Safer with complex CSVs / mixed types
            on_bad_lines='skip'
        )
    except FileNotFoundError:
        print(f"Error: The file was not found at {csv_path}")
        return
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return

    print("Spalten im DataFrame:", df.columns.tolist())

    # --- Data Cleaning and Preparation ---
    # Clean 'reply_to_id' to use pd.NA for missing values for consistent checking
    # common string representations of NaN/None when read as str dtype
    df['reply_to_id'] = df['reply_to_id'].replace({'nan': pd.NA, '': pd.NA, None: pd.NA, 'None': pd.NA})
    # Ensure tweet_id is string and suitable for map keys (already str by dtype)
    df['tweet_id'] = df['tweet_id'].astype(str)


    # Remove duplicates based on 'tweet_id'
    initial_rows = len(df)
    df.drop_duplicates(subset=['tweet_id'], inplace=True, keep='first')
    print(f"Anfängliche Anzahl Zeilen: {initial_rows}, nach Duplikaten entfernen: {len(df)}")
    df = df[df['retweeted_user_ID'].isna()]
    print(f"Anzahl Zeilen nach dem Entfernen von Retweets: {len(df)} (reduziert um {initial_rows-len(df)})")
    # 2. Nur-Emoji-Posts entfernen
    # Wir wenden die Hilfsfunktion auf die 'full_text'-Spalte an
    df = df[~df['full_text'].apply(is_only_emojis)]
    print(f"Anzahl Zeilen nach Entfernen von Nur-Emoji-Posts: {len(df)}  (reduziert um {initial_rows-len(df)})")

    # 3. Context-aware URL filtering - Replace the old is_only_link function
    # Remove posts with URLs unless they have substantial context
    #df = df[~df['full_text'].apply(is_only_link)]
    df = df[~df['full_text'].apply(contains_link)]
    print(f"Anzahl Zeilen nach Entfernen von Nur-Link-Posts: {len(df)}  (reduziert um {initial_rows-len(df)})")

    #4. Remove posts with less than 3 words
    df = df[df['full_text'].apply(has_minimum_words)]
    print(f"Anzahl Zeilen nach Entfernen von Posts mit weniger als 3 Wörtern: {len(df)} (reduziert um {initial_rows-len(df)})")



    if df.empty:
        print("DataFrame is empty after removing duplicates. Exiting.")
        return

    # Create a lookup map for tweet details (used for 'previous_message')
    # This map uses the string version of tweet_id as keys.
    tweet_details_map = df.set_index('tweet_id')[['created_at', 'screen_name', 'full_text']].to_dict(orient='index')

    # --- User Filtering ---
    # Ensure 'original_user_id' is not null before counting
    df_valid_users = df[df['original_user_id'].notna()]
    original_tweet_counts = df_valid_users['original_user_id'].value_counts()
    
    replies_df_all = df_valid_users[df_valid_users['reply_to_id'].notnull()]
    reply_counts = replies_df_all['original_user_id'].value_counts()

    eligible_users = []
    user_stats = []

    print("Filtering eligible users...")
    for user_id_str in original_tweet_counts.index:
        total_posts = original_tweet_counts.get(user_id_str, 0)
        num_replies = reply_counts.get(user_id_str, 0)
        num_originals = total_posts - num_replies

        if num_replies >= (2 * TARGET_REPLIES_PER_SET) and \
           num_originals >= (2 * TARGET_ORIGINALS_PER_SET) and \
           total_posts >= (2 * TARGET_POSTS_PER_SET): # Ensure enough total posts for two sets
            eligible_users.append(user_id_str)
            user_stats.append({
                "user_id": user_id_str,
                "total_tweets_by_user": int(total_posts), # Renamed for clarity
                "replies_by_user": int(num_replies),
                "original_tweets_by_user": int(num_originals) # Non-replies
            })
    
    print(f"Anzahl geeigneter Nutzer gefunden: {len(eligible_users)}")
    if not eligible_users:
        print("Keine geeigneten Nutzer gefunden basierend auf den Kriterien. Beende Skript.")
        return

    # --- Prepare Output Directory ---
    output_dir = Path("data/filtered_users_without_links")
    output_dir.mkdir(parents=True, exist_ok=True)

    # --- Helper function to get previous message details ---
    def get_previous_message_details(reply_row_series, tdm):
        original_tweet_id = reply_row_series['reply_to_id'] # This should be a string ID now
        if pd.notnull(original_tweet_id):
            # original_tweet_id is already a string from dtype and cleaning
            message_data = tdm.get(original_tweet_id)
            if message_data:
                # Ensure screen_name is handled if it's missing or NaN from the source
                screen_name_val = message_data.get('screen_name')
                formatted_screen_name = str(screen_name_val) if pd.notnull(screen_name_val) else ''
                
                # Format as a single string: "Date, Author: Message"
                created_at = message_data.get('created_at', '')
                full_text = message_data.get('full_text', '')
                
                formatted_message = f"{created_at}, {formatted_screen_name}: {full_text}"
                return formatted_message
        return None

    # --- Process Each Eligible User ---
    processed_user_count = 0
    for user_id_str in eligible_users:
        print(f"Processing user: {user_id_str}")
        user_df = df[df['original_user_id'] == user_id_str].copy()
        
        # Shuffle all user's tweets once
        user_df = user_df.sample(frac=1, random_state=42)

        all_user_replies = user_df[user_df['reply_to_id'].notnull()].copy()
        all_user_originals = user_df[user_df['reply_to_id'].isnull()].copy()

        # Check again if enough material after splitting (should be guaranteed by initial filter)
        if not (len(all_user_replies) >= (2 * TARGET_REPLIES_PER_SET) and \
                len(all_user_originals) >= (2 * TARGET_ORIGINALS_PER_SET)):
            print(f"  Skipping user {user_id_str}: Insufficient replies/originals after splitting (unexpected).")
            continue
            
        # Create Holdout Set
        holdout_replies = all_user_replies.head(TARGET_REPLIES_PER_SET)
        holdout_originals = all_user_originals.head(TARGET_ORIGINALS_PER_SET)
        holdout_df = pd.concat([holdout_replies, holdout_originals]).sample(frac=1, random_state=42) # Shuffle combined parts

        # Create History Set (from parts not used in holdout)
        history_replies = all_user_replies.iloc[TARGET_REPLIES_PER_SET : 2 * TARGET_REPLIES_PER_SET]
        history_originals = all_user_originals.iloc[TARGET_ORIGINALS_PER_SET : 2 * TARGET_ORIGINALS_PER_SET]
        history_df = pd.concat([history_replies, history_originals]).sample(frac=1, random_state=42)


        # Validate set composition
        valid_sets = True
        if not (len(holdout_df) == TARGET_POSTS_PER_SET and \
                len(holdout_df[holdout_df['reply_to_id'].notnull()]) == TARGET_REPLIES_PER_SET):
            print(f"  Skipping user {user_id_str}: Holdout set composition incorrect. Size: {len(holdout_df)}, Replies: {len(holdout_df[holdout_df['reply_to_id'].notnull()])}")
            valid_sets = False
        
        if not (len(history_df) == TARGET_POSTS_PER_SET and \
                len(history_df[history_df['reply_to_id'].notnull()]) == TARGET_REPLIES_PER_SET):
            print(f"  Skipping user {user_id_str}: History set composition incorrect. Size: {len(history_df)}, Replies: {len(history_df[history_df['reply_to_id'].notnull()])}")
            valid_sets = False
        
        if not valid_sets:
            continue

        # Add 'previous_message' to replies in both sets
        for current_set_df in [history_df, holdout_df]:
            current_set_df['previous_message'] = None # Initialize column
            reply_mask = current_set_df['reply_to_id'].notnull()
            
            # Ensure apply is only on rows where reply_mask is True and avoid issues with empty slices
            if reply_mask.any():
                 current_set_df.loc[reply_mask, 'previous_message'] = current_set_df[reply_mask].apply(
                    lambda row: get_previous_message_details(row, tweet_details_map), axis=1
                )
        
        # Save user's data to JSONL
        output_path = output_dir / f"{user_id_str}.jsonl"
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                # History set
                history_records = history_df.to_dict(orient="records")
                json.dump({"user_id": user_id_str, "set": "history", "tweets": history_records}, f)
                f.write('\n')
                
                # Holdout set
                holdout_records = holdout_df.to_dict(orient="records")
                json.dump({"user_id": user_id_str, "set": "holdout", "tweets": holdout_records}, f)
                f.write('\n')
            processed_user_count += 1
            print(f"  Successfully processed and saved data for user {user_id_str}")
        except Exception as e:
            print(f"  Error saving data for user {user_id_str}: {e}")

    print(f"\nFinished processing. Total users processed and saved: {processed_user_count}")

    # --- Save Global Metadata ---
    meta_dir = Path("output_directory_v2")
    meta_dir.mkdir(parents=True, exist_ok=True) # Ensure base output_directory exists

    final_user_stats_path = meta_dir / "user_stats.json"
    with open(final_user_stats_path, 'w', encoding='utf-8') as f:
        json.dump(user_stats, f, indent=4)
    print(f"User statistics saved to: {final_user_stats_path}")

    # Save only the list of users for whom files were actually created (or attempted successfully)
    # For simplicity, we are saving the 'eligible_users' list based on initial criteria.
    # A more precise list would be users for whom files were actually written.
    eligible_users_path = meta_dir / "eligible_users.json"
    with open(eligible_users_path, 'w', encoding='utf-8') as f:
        json.dump(eligible_users, f, indent=4) # This is the list of users meeting the criteria for set creation
    print(f"List of eligible users saved to: {eligible_users_path}")

if __name__ == '__main__':
    process_tweets(path)

Loading data from: data/raw/Kopie von FolloweeIDs2_tweets_df_AugustPull.csv
Spalten im DataFrame: ['full_text', 'tweet_id', 'created_at', 'screen_name', 'original_user_id', 'retweeted_user_ID', 'collected_at', 'reply_to_id', 'reply_to_user', 'expandedURL']
Anfängliche Anzahl Zeilen: 7790741, nach Duplikaten entfernen: 6900898
Anzahl Zeilen nach dem Entfernen von Retweets: 4728900 (reduziert um 3061841)
Anzahl Zeilen nach Entfernen von Nur-Emoji-Posts: 4727989  (reduziert um 3062752)
Anzahl Zeilen nach Entfernen von Nur-Link-Posts: 2169889  (reduziert um 5620852)
Anzahl Zeilen nach Entfernen von Posts mit weniger als 3 Wörtern: 2023786 (reduziert um 5766955)
Filtering eligible users...
Anzahl geeigneter Nutzer gefunden: 25
Processing user: 1.0679687024008764e+18
  Successfully processed and saved data for user 1.0679687024008764e+18
Processing user: 1.470492384274309e+18
  Successfully processed and saved data for user 1.470492384274309e+18
Processing user: 38283342.0
  Successfully pro